### **Import Module**

In [1]:
import re
import pandas as pd
from bs4 import BeautifulSoup as bs
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import word_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer
from transformers import pipeline
import tensorflow as tf
import warnings
import mysql.connector
import os
from dotenv import load_dotenv

#### **Suppress the specific FutureWarning**

In [2]:
warnings.filterwarnings("ignore", category=FutureWarning, module="transformers.tokenization_utils_base")
warnings.filterwarnings('ignore',message='The name tf.losses.sparse_softmax_cross_entropy is deprecated')
warnings.filterwarnings('ignore',message='The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
warnings.filterwarnings('ignore', category=DeprecationWarning)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

### **All Product URL :-**

In [3]:
Product_URL_01 = "https://www.flipkart.com/fitoda-fashion-men-graphic-print-casual-multicolor-shirt/p/itmb1e9ad9ec3320?pid=SHTGKW43PPG4MYEM&lid=LSTSHTGKW43PPG4MYEMKKK7UK&marketplace=FLIPKART&fm=productRecommendation%2Fsimilar&iid=R%3As%3Bp%3ASHTGFQVTBGSDQHNY%3Bl%3ALSTSHTGFQVTBGSDQHNYKBCECM%3Bpt%3App%3Buid%3A61e32bf4-7bcb-11ef-b7fb-975dfaf6ad6e%3B.SHTGKW43PPG4MYEM&ppt=pp&ppn=pp&ssid=lntb3u2ebk0000001727329869450&otracker=pp_reco_Similar%2BProducts_1_34.productCard.PMU_HORIZONTAL_fitoda%2Bfashion%2BMen%2BGraphic%2BPrint%2BCasual%2BMulticolor%2BShirt_SHTGKW43PPG4MYEM_productRecommendation%2Fsimilar_0&otracker1=pp_reco_PINNED_productRecommendation%2Fsimilar_Similar%2BProducts_GRID_productCard_cc_1_NA_view-all&cid=SHTGKW43PPG4MYEM"
product_URL_02 = "https://www.flipkart.com/marmic-fab-men-printed-casual-white-black-shirt/p/itmd14749f2319a7?pid=SHTGJXUJWGNH7KZ7&lid=LSTSHTGJXUJWGNH7KZ7OPLKGR&marketplace=FLIPKART&sattr[]=color&st=color"
product_URL_03 = "https://www.flipkart.com/combraided-men-solid-casual-light-blue-shirt/p/itm7331f7b6e2618?pid=SHTG6AGZUNZRJY2H&lid=LSTSHTG6AGZUNZRJY2HS3L2RZ&marketplace=FLIPKART&store=clo%2Fash&srno=b_1_2&otracker=browse&otracker1=hp_rich_navigation_PINNED_neo%2Fmerchandising_NA_NAV_EXPANDABLE_navigationCard_cc_1_L2_view-all&fm=organic&iid=en_sZ5f25YUi1yLU5aq0hfKLL5JeCOq2ITn5kSMbYh7vpQ_ETLH_df6VOOs-oo7t4aI1L5k2lqY-9gm-RNnBLn7vQ%3D%3D&ppt=browse&ppn=browse&ssid=olt3rp9t3k0000001727359632948"
product_URL_04 = "https://www.flipkart.com/y-company-casual-self-design-women-white-top/p/itmc45623bf375df?pid=TOPGZKQ2HHEXGYAC&lid=LSTTOPGZKQ2HHEXGYACHURA7W&marketplace=FLIPKART&q=beach+wear+tops+for+women&store=clo%2Fash%2Fohw%2F36j&spotlightTagId=BestsellerId_clo%2Fash%2Fohw%2F36j&srno=s_1_1&otracker=AS_Query_OrganicAutoSuggest_7_4_na_na_ps&otracker1=AS_Query_OrganicAutoSuggest_7_4_na_na_ps&fm=search-autosuggest&iid=b0788323-36ac-41bc-b016-119708205e33.TOPGZKQ2HHEXGYAC.SEARCH&ppt=sp&ppn=sp&ssid=fxybozal680000001727363123575&qH=a5683c78cacfd0b6"
product_URL_05 = ""
product_URL_06 = ""
product_URL_07 = ""
product_URL_08 = ""
product_URL_09 = ""
product_URL_10 = ""
product_URL_11 = ""
product_URL_12 = ""

### **Product Details** 

In [5]:
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Run the browser in headless mode
options.add_argument("--disable-gpu")
driver = webdriver.Chrome(options=options)
driver.get(product_URL_02)
time.sleep(0)  # Let the page load

# Scroll down to load more Details (adjust the number of scrolls as needed)
scrolls = 3
for _ in range(scrolls):
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
    time.sleep(0)
# Get the page source after scrolling
Product_Page = driver.page_source
# Close the browser
driver.quit()
# Now you can use BeautifulSoup to parse the loaded content
HTML = bs(Product_Page, 'html.parser')

Product_Name = HTML.find("span",{"class":"VU-ZEz"}).text.replace('\xa0', ' ').strip()
Product_Image = HTML.find("img",{"class":"_53J4C- utBuJY"}).get('src')
Hover_Image = HTML.find("div", {"class": "Pz+aTd"}).img.get('src')
Original_Price = float(HTML.find('div', class_=['yRaY8j', 'A6+E6v']).get_text().strip().replace('₹', '').replace(',', '').strip())
Discount = float(HTML.find("div",{"class":"UkUFwK WW8yVX dB67CR"}).text.replace("% off",""))
Special_Price = float(HTML.find("div",{"class":"Nx9bqj CxhGGd"}).text.replace('₹','').replace(',',''))
Product_Rating = float(HTML.find("div",{"class":"XQDdHH"}).text)
Total_Number_of_Ratings = int((re.findall(r'\d{1,3}(?:,\d{3})*', HTML.find_all("span",{"class":"Wphh3N"})[0].span.text))[0].replace(',', ''))
Total_Number_of_Reviews = int((re.findall(r'\d{1,3}(?:,\d{3})*', HTML.find_all("span",{"class":"Wphh3N"})[0].span.text))[1].replace(',', ''))
Delivery_Date = HTML.find('span', class_='Y8v7Fl').text.split(",")[0]
Seller_Name = HTML.find("div",{"class":"yeLeBC"}).span.span.text
Seller_Rating = float(HTML.find("div",{"class":"yeLeBC"}).div.text)
Link = HTML.find("div",{"class":"col pPAw9M iIbIvC"}).find_all('a')[-1].get('href')

# Return the product details in a dictionary
Product_Detail = {
    "Product URL": Product_URL_01,
    "Product Name": Product_Name,
    "Product Image": Product_Image,
    "Hover Image": Hover_Image,
    "Original Price": Original_Price,
    "Discount %": Discount,
    "Special Price": Special_Price,
    "Rating": Product_Rating,
    "Total Number of Ratings": Total_Number_of_Ratings,
    "Total Number of Reviews": Total_Number_of_Reviews,
    "Delivery Date": Delivery_Date,
    "Seller Name": Seller_Name,
    "Seller Rating": Seller_Rating,
    "Link": Link
}

In [7]:
Reviews_Page_URL = [f"https://www.flipkart.com{Product_Detail['Link']}&page={i}" for i in range(1, 11)]
Buyer_Name = []
Rating = []
Comment = []

for i in Reviews_Page_URL:
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Run the browser in headless mode
    options.add_argument("--disable-gpu")
    driver = webdriver.Chrome(options=options)
    driver.get(i)
    time.sleep(0)  # Let the page load
    scrolls = 3
    for _ in range(scrolls):
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
        time.sleep(0)
    Page = driver.page_source
    driver.quit()
    Reviews_HTML = bs(Page, 'html.parser')

    for i in Reviews_HTML.find_all("p",{"class":"_2NsDsF AwS1CA MDcJkH"}):
        Buyer_Name.append(i.text)

    for i in Reviews_HTML.find_all("div", {"class": "cPHDOP col-12-12"})[2:-1]:
        first_class = i.find("div", {"class": "XQDdHH Ga3i8K _9lBNRY"})
        second_class = i.find("div", {"class": "XQDdHH Czs3gR Ga3i8K _9lBNRY"})
        third_class = i.find("div", {"class": "XQDdHH Js30Fc Ga3i8K _9lBNRY"})
        if first_class:
            Rating.append(int(first_class.text))
        elif second_class:
            Rating.append(int(second_class.text))
        elif third_class:
            Rating.append(int(third_class.text))
            
    for i in Reviews_HTML.find_all("div",{"class":"_11pzQk"}):
        Comment.append(i.text)

dict = {
    "Name" : Buyer_Name,
    "Rating" : Rating,
    "Comment" : Comment
}

df = pd.DataFrame(dict)

In [8]:
def clean_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation and numbers
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(filtered_tokens)

In [9]:
def get_sentiment(text):
    sia = SentimentIntensityAnalyzer()
    sentiment = sia.polarity_scores(text)
    if sentiment['compound'] >= 0.05:
        return 'positive'
    elif sentiment['compound'] <= -0.05:
        return 'negative'
    else:
        return 'neutral'

In [10]:
def classify_sentiment(text):
    # Load sentiment analysis pipeline
    sentiment_pipeline = pipeline('sentiment-analysis', model='distilbert-base-uncased-finetuned-sst-2-english')
    result = sentiment_pipeline(text)
    return result[0]['label']

In [11]:
def process():
    df['cleaned_review'] = df['Comment'].apply(clean_text)
    df['tokens'] = df['cleaned_review'].apply(word_tokenize)
    df['sentiment'] = df['cleaned_review'].apply(get_sentiment)
    df['sentiment'] = df['Comment'].apply(classify_sentiment)
    Positive_Review = round(((df[df['sentiment'] == 'POSITIVE'].shape[0])/100)*100)
    Negative_Review = round(((df[df['sentiment'] == 'NEGATIVE'].shape[0])/100)*100)
    Neutral_Review = round(((df[df['sentiment'] == 'NEUTRAL'].shape[0])/100)*100)
    Product_Detail['Positive Reviews %'] = Positive_Review
    Product_Detail['Negative Reviews %'] = Negative_Review
    Product_Detail['Neutral Reviews %'] = Neutral_Review
    return Product_Detail

In [12]:
Product_Detail = process()

In [14]:
# Load environment variables from .env file
load_dotenv()

# Establish database connection
connection = mysql.connector.connect(
    host=os.getenv('MYSQL_HOST'),
    user=os.getenv('MYSQL_USER'),
    password=os.getenv('MYSQL_PASSWORD'),
    database=os.getenv('MYSQL_DB')
)

cursor = connection.cursor()

# Create table if it doesn't exist
cursor.execute('''
    CREATE TABLE IF NOT EXISTS product_details (
        id INT AUTO_INCREMENT PRIMARY KEY,
        product_url TEXT,
        product_name TEXT,
        product_image TEXT,
        hover_image TEXT,
        original_price FLOAT,
        discount FLOAT,
        special_price FLOAT,
        overall_rating FLOAT,
        total_ratings INT,
        total_reviews INT,
        expected_delivery_date TEXT,
        seller_name TEXT,
        seller_rating FLOAT,
        positive_reviews_percentage FLOAT,
        negative_reviews_percentage FLOAT,
        neutral_reviews_percentage FLOAT
    )
''')

# Ensure all values are in the correct format
for key, value in Product_Detail.items():
    if isinstance(value, list):  # Convert list to string
        Product_Detail[key] = ', '.join(map(str, value))
    elif value == '' or value is None:  # Handle empty or None values
        Product_Detail[key] = None
    elif key in ["Original Price", "Discount %", "Special Price", "Rating", "Seller Rating",
                 "Positive Reviews %", "Negative Reviews %", "Neutral Reviews %"]:
        # Convert numeric fields to float
        Product_Detail[key] = float(value) if value not in [None, ''] else None
    elif key in ["Total Number of Ratings", "Total Number of Reviews"]:
        # Convert to integers
        Product_Detail[key] = int(value) if value not in [None, ''] else None

# Insert product details into the database
insert_query = '''
    INSERT INTO product_details (
        product_url, product_name, product_image, hover_image, original_price, discount, special_price, overall_rating, 
        total_ratings, total_reviews, expected_delivery_date, seller_name, seller_rating, 
        positive_reviews_percentage, negative_reviews_percentage, neutral_reviews_percentage
    ) 
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    '''

# Execute the query
cursor.execute(insert_query, (
    Product_Detail.get("Product URL"), Product_Detail.get("Product Name"), Product_Detail.get("Product Image"),
    Product_Detail.get("Hover Image"), Product_Detail.get("Original Price"), Product_Detail.get("Discount %"),
    Product_Detail.get("Special Price"), Product_Detail.get("Rating"), Product_Detail.get("Total Number of Ratings"),
    Product_Detail.get("Total Number of Reviews"), Product_Detail.get("Delivery Date"), Product_Detail.get("Seller Name"),
    Product_Detail.get("Seller Rating"), Product_Detail.get("Positive Reviews %"), Product_Detail.get("Negative Reviews %"),
    Product_Detail.get("Neutral Reviews %")
))

# Commit the transaction
connection.commit()

# Close the cursor and connection
cursor.close()
connection.close()